In [1]:
def generatePrime(numprimes):
    count = 0
    potentialprime = 2
    primes=[]
    
    def primetest(potentialprime):
        divisor = 2
        while divisor <= potentialprime:
            if potentialprime == 2:
                return True
            elif potentialprime % divisor == 0:
                return False
                break
            while potentialprime % divisor != 0:
                if potentialprime - divisor > 1:
                    divisor += 1
                else:
                    return True
    
    while count < numprimes:
        if primetest(potentialprime) == True:
            primes.append((count+1,potentialprime))
            count += 1
            potentialprime += 1
        else:
            potentialprime += 1
            
    return primes




import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 0
    
pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(safeDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(abs,1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1000,1000))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = (round(func(x[0]) - x[1])**2 for x in points)
    return math.sqrt(math.fsum(sqerrors)),
#generating 1st nth prime number
points=generatePrime(50)

toolbox.register("evaluate", evalSymbReg, points=points)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))


def main():
    random.seed(318)

    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.7, 0.1, 100, stats=mstats,halloffame=hof, verbose=True)
    
    #print('Best individual : ', hof[0], hof[0].fitness)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    p,l,h=main()
    

print('Best individual : ', h[0], h[0].fitness)
f1=gp.compile(h[0],pset)
def f(x):
    global points
    print(points[x][1])
    print(f1(x))

def diff():
    global points
    for x in points:
        print(x[1],'-',round(f1(x[0])),'=',x[1]-round(f1(x[0])))

   	      	                      fitness                      	              size             
   	      	---------------------------------------------------	-------------------------------
gen	nevals	avg       	max        	min    	std        	avg  	max	min	std    
0  	500   	9.9988e+07	4.88803e+10	447.952	2.18413e+09	3.468	7  	2  	1.51161
1  	346   	2256.89   	284957     	261.14 	14890.6    	3.416	14 	1  	1.85228
2  	385   	3.46548e+06	1.72795e+09	388.268	7.71988e+07	3.498	13 	1  	1.86279
3  	349   	9213.4     	3.64785e+06	388.268	163357     	3.374	14 	1  	2.05186
4  	359   	1222.66    	179830     	388.268	8194.76    	3.322	21 	1  	2.16664
5  	376   	2663.32    	553667     	388.268	26351.9    	3.244	14 	1  	1.7206 
6  	357   	333832     	1.64458e+08	387.969	7.34749e+06	3.358	17 	1  	1.59932
7  	353   	34230.4    	1.60939e+07	261.14 	719138     	3.648	17 	1  	1.81331
8  	368   	7773.83    	2.42681e+06	261.14 	113524     	3.8  	15 	1  	1.82975
9  	348   	22845.2    	5.09094e+06	261.14 	

In [2]:
diff()

2 - 3 = -1
3 - 6 = -3
5 - 9 = -4
7 - 12 = -5
11 - 15 = -4
13 - 18 = -5
17 - 21 = -4
19 - 24 = -5
23 - 27 = -4
29 - 30 = -1
31 - 33 = -2
37 - 36 = 1
41 - 39 = 2
43 - 42 = 1
47 - 45 = 2
53 - 49 = 4
59 - 55 = 4
61 - 62 = -1
67 - 67 = 0
71 - 71 = 0
73 - 74 = -1
79 - 78 = 1
83 - 84 = -1
89 - 91 = -2
97 - 97 = 0
101 - 102 = -1
103 - 105 = -2
107 - 109 = -2
109 - 114 = -5
113 - 121 = -8
127 - 127 = 0
131 - 132 = -1
137 - 135 = 2
139 - 139 = 0
149 - 144 = 5
151 - 150 = 1
157 - 157 = 0
163 - 162 = 1
167 - 166 = 1
173 - 169 = 4
179 - 173 = 6
181 - 180 = 1
191 - 187 = 4
193 - 192 = 1
197 - 197 = 0
199 - 200 = -1
211 - 203 = 8
223 - 209 = 14
227 - 216 = 11
229 - 222 = 7


In [3]:
expr = h[0]
print(expr)
nodes, edges, labels = gp.graph(expr)

### Graphviz Section ###
import pygraphviz as pgv

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")

add(add(-15, x), add(add(add(x, abs(add(cos(x), add(-15, x)))), add(cos(x), add(cos(safeDiv(753, -686)), x))), x))
